In [ ]:
# In addition to the topic models I also made a small n-gram viewer app inspired by Google Books N-gram Viewer. This is the 
# script to prepare the n-gram combinations for that app 

In [1]:
import pandas as pd
from spacy.lang.bg.stop_words import STOP_WORDS as BG_STOPWORDS
from spacy.lang.en import stop_words
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 
import nltk
nltk.download('punkt')
import string
import re
import spacy
from collections import Counter

bulgarian_df = pd.read_csv("./data/bulgarian_submissions.csv")
bulgarian = pd.read_csv("./data/bulgarian_comments.csv")
result_df = pd.concat([bulgarian, bulgarian_df], axis=0, ignore_index=True)

# Let's get only posts starting 2018, where bulgarian posts start to increase, otherwise due
# to the low number of activity in first 10 years we would have flat line for almost all words until 
# 2018-2019
result_df = result_df[result_df['year_month'] >= '2018-01']

/var/folders/w8/bk8xzzt10l30g9c3zpfjjh700000gn/T/ipykernel_62102/173051064.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  bulgarian = pd.read_csv("./data/bulgarian_comments.csv")


In [3]:
## First quick and drity attempt at getting n-grams
stopwords_bg = 'amp иначе xb gt та ама на от за да се по са ще че не това си като до през които най при но има след който към бъде той още може му което много със която или само тази те обаче във вече около както над така между ако лв им тези преди млн бе също пред ни когато защото кв би пък тъй ги ли пак според този все някои не'
stopwords_custom = stopwords_bg.split()
# stopwords_custom.append('не')

# add appropriate words that will be ignored in the analysis
ADDITIONAL_STOPWORDS = list(BG_STOPWORDS) + list(stop_words.STOP_WORDS) + stopwords_custom

def prepare_ngram(text, ngrams):
    # Take punctuations out
    cleaned_string = re.sub('\[,.*?“”…\]', '', text)
    cleaned_string = re.sub(r'[“”]', '', cleaned_string)

    # Remove any digits
    cleaned_string = ''.join([i for i in cleaned_string if not i.isdigit()])

    # Tokenise the data
    cleaned_string = re.sub('[%s]' % re.escape(string.punctuation), ' ', cleaned_string)

    # Why did I needed to do lowercase? Double-check that - it doesn't match Google Ngram behavior
    cleaned_string = cleaned_string.lower()
    TOKENS = word_tokenize(cleaned_string) 

    # Filter those stop words out
    filtered_sentence = []
    
    for w in TOKENS: 
        if w not in ADDITIONAL_STOPWORDS:
            filtered_sentence.append(w)
    
    # Count phrases
    gram_df = pd.Series(nltk.ngrams(filtered_sentence, ngrams)).value_counts()
    return gram_df

[nltk_data] Downloading package punkt to /Users/ivo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("bg_news_lg")

# Define your list of additional custom stop words
stopwords_bg = 'amp the иначе xb gt та ама на от за да се по са ще че не това си като до през които най при но има след който към бъде той още може му което много със която или само тази те обаче във вече около както над така между ако лв им тези преди млн бе също пред ни когато защото кв би пък тъй ги ли пак според този все някои не'
CUSTOM_STOPWORDS = stopwords_bg.split()

# Add custom stop words to spaCy's default stop words set
for word in CUSTOM_STOPWORDS:
    nlp.Defaults.stop_words.add(word)
    nlp.vocab[word].is_stop = True

def prepare_ngram_spacy(text, ngrams):
    # Process the text with spaCy
    doc = nlp(text)

    # Filter out stop words and punctuation
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    tokens = [token.lower() for token in tokens]
    tokens = [token.replace("-(се)", "") for token in tokens]

    # Generate n-grams
    n_grams = zip(*[tokens[i:] for i in range(ngrams)])
    n_grams = [' '.join(n_gram) for n_gram in n_grams]

    # Count phrases
    ngram_counts = Counter(n_grams)
    ngram_series = pd.Series(ngram_counts).sort_values(ascending=False)
    return ngram_series


def prepare_bigram_spacy(text, ngrams):
    # Process the text with spaCy
    doc = nlp(text)

    # Filter out stop words and punctuation
    tokens = [token.lower_ for token in doc if not token.is_stop and token.is_alpha]

    # Generate n-grams
    n_grams = zip(*[tokens[i:] for i in range(ngrams)])
    n_grams = [' '.join(n_gram) for n_gram in n_grams]

    # Count phrases
    ngram_counts = Counter(n_grams)
    ngram_series = pd.Series(ngram_counts).sort_values(ascending=False)
    return ngram_series

In [5]:
text = "Ако реша, че ми е от полза, няма да се колебая и за секунда. В момента съм на мнение, че най-много ми е от полза отдалечаване от Русия, затова гласувам с надеждата, че зад ппдб наистина стои американското посолство. Тук вече идват различните мнения - според теб е защото Прокопиев така ми е казал, според мен е мнение, градено с десетилетия в моя съзнателен живот. Let's agree to disagree. \n\nАйде весело посрещане и успешна и здрава година. Местя се в ЕС."
prepare_ngram_spacy(text, 2)

реша полза               1
agree to                 1
мнение градя             1
градя десетилетие        1
десетилетие мой          1
мой съзнателен           1
съзнателен let           1
let agree                1
to disagree              1
прокопиев кажа           1
disagree айде            1
айде весел               1
весел посрещане          1
посрещане успешен        1
успешен здрав            1
здрав местя              1
кажа мнение              1
аз прокопиев             1
полза колебая            1
гласувам надежда         1
колебая секунда          1
секунда мнение           1
мнение полза             1
полза отдалечаване       1
отдалечаване русия       1
русия гласувам           1
надежда ппдб             1
мнение аз                1
ппдб стоя                1
стоя американски         1
американски посолство    1
посолство идвам          1
идвам различен           1
различен мнение          1
местя ес                 1
dtype: int64

In [ ]:
# Create a full dataset for unigrams
full_df_unigram = result_df.groupby('created_utc').apply(lambda x: prepare_ngram_spacy(x['body'].str.cat(sep=', '), 1))
full_df_unigram = full_df_unigram.reset_index().rename(columns={'created_utc' : 'date', 'level_1' : 'unigram', 0 : 'count'})
full_df_unigram = full_df_unigram.explode('unigram')

In [ ]:
full_df_bigram = result_df.groupby('created_utc').apply(lambda x: prepare_bigram_spacy(x['body'].str.cat(sep=', '), 2))
full_df_bigram = full_df_bigram.reset_index().rename(columns={'created_utc' : 'date', 'level_1' : 'bigram', 0 : 'count'})
full_df_bigram = full_df_bigram.explode('bigram')

full_df_bigram = full_df_bigram[full_df_bigram['bigram'].str.len() > 2]
# full_df_bigram = full_df_bigram.drop(columns=["index"])

In [ ]:
# Prepare monthly counts and proportions for the combined dataframe (we want to be able to compare
# unigrams to bigrams directly)
full_df_unigram = full_df_unigram[full_df_unigram['unigram'].str.len() > 1]
full_df_unigram.set_index('date', inplace=True)
full_df_unigram.index = pd.DatetimeIndex(full_df_unigram.index)
monthly_unigram = full_df_unigram.groupby([pd.Grouper(freq="M"), "unigram"]).sum().reset_index()
monthly_unigram['ratio'] = (monthly_unigram.groupby(['unigram','date'])['count'].transform(sum) / monthly_unigram.groupby('date')['count'].transform(sum))
monthly_unigram = monthly_unigram.rename(columns={"unigram" : "gram"})

In [ ]:
full_df_bigram.set_index('date', inplace=True)
full_df_bigram.index = pd.DatetimeIndex(full_df_bigram.index)
monthly_bigram = full_df_bigram.groupby([pd.Grouper(freq="M"), "bigram"]).sum().reset_index()
monthly_bigram['ratio'] = (monthly_bigram.groupby(['bigram','date'])['count'].transform(sum) / monthly_bigram.groupby('date')['count'].transform(sum))
monthly_bigram = monthly_bigram.rename(columns={"bigram" : "gram"})

In [ ]:
full_df_unigram.groupby("unigram").sum().reset_index().sort_values(by=["count"], ascending=False).head(30)

In [ ]:
# let's filter out everything that is only mentioned 4 times?
grouped_bigram = full_df_bigram.groupby("bigram").sum().reset_index().sort_values(by=["count"], ascending=False)
grouped_bigram = grouped_bigram[grouped_bigram['count'] > 9]
keep_bigram = grouped_bigram['bigram'].values.tolist()
monthly_bigram = monthly_bigram[monthly_bigram['gram'].isin(keep_bigram)]

In [ ]:
# let's filter out everything that is only mentioned 4 times?
grouped_unigram = full_df_unigram.groupby("unigram").sum().reset_index().sort_values(by=["count"], ascending=False)
grouped_unigram = grouped_unigram[grouped_unigram['count'] > 7]
keep_unigram = grouped_unigram['unigram'].values.tolist()
monthly_unigram = monthly_unigram[monthly_unigram['gram'].isin(keep_unigram)]

In [ ]:
monthly_unigram.to_csv("./data/unigram_full_df.csv")
monthly_bigram.to_csv("./data/bigram_full_df.csv")